## Imports

In [1]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import time
from datetime import timedelta
import math

import h5py

%matplotlib inline 

/Users/steven/.local/share/virtualenvs/B4-5100-4F18-LSDA-iftAmOns/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load Data

In [2]:
d_train = h5py.File('../images/img28_bw/train_28_bw.h5', 'r')
d_test = h5py.File('../images/img28_bw/test_28_bw.h5', 'r')
d_val = h5py.File('../images/img28_bw/validation_28_bw.h5', 'r')

In [3]:
Xtrain = d_train['train']['images']
ytrain = d_train['train']['labels']

Xtest = d_test['test']['images']

Xval = d_val['validation']['images']
yval = d_val['validation']['labels']

print(\
'Training samples:\t{}\n\
Test samples:\t\t{}\n\
Validation Samples:\t{}'.format(Xtrain.shape[0], Xtest.shape[0], Xval.shape[0]))

Training samples:	192171
Test samples:		12704
Validation Samples:	6309


## Reshape Labels to One-Hot

In [4]:
def flatImage(data, start, end):
    tmp = data[start:end, :].reshape(-1, Xtrain[start:end, :].shape[0])
    return(np.transpose(tmp))

In [5]:
# ytrain = np.eye(ytrain.shape[0], 128)[ytrain.value]
# yval   = np.eye(yval.shape[0], 128)[yval.value]
ytrain = np.reshape(ytrain.value, (-1, 1))
yval = np.reshape(yval.value, (-1, 1))

### Flags

In [6]:
maxIter = 3000
lr = 0.001

In [7]:
# Input dimension
in_dim = Xtrain.shape[1]**2

In [8]:
# Initialize placeholders
x = tf.placeholder(shape=[None, in_dim], dtype=tf.float32, name='input')
y = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='target')

In [9]:
# TensorBoard will collapse the following nodes
with tf.name_scope('model') as scope:
    # Create variables for logistic regression
    A = tf.Variable(tf.random_normal(shape=[in_dim,1]))
    b = tf.Variable(tf.random_normal(shape=[1,1]))

    # Declare model operations
    model_output = tf.add(tf.matmul(x, A), b)

In [10]:
# Declare loss function
with tf.name_scope('loss') as scope:
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_output, labels=y))
    tf.summary.scalar('cross-entropy', loss)

In [11]:
# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(lr)
train_step = my_opt.minimize(loss)

In [12]:
# Map model output to binary predictions
with tf.name_scope('binary_prediction') as scope:
    prediction = tf.round(tf.sigmoid(model_output))
with tf.name_scope('0-1-loss') as scope:
    predictions_correct = tf.cast(tf.equal(prediction, y), tf.float32)
    accuracy = tf.reduce_mean(predictions_correct)
    tf.summary.scalar('accuracy', accuracy)

In [13]:
with tf.Session() as sess:
    # Logging
    merged = tf.summary.merge_all()  # collect all summaries in the graph
    train_writer = tf.summary.FileWriter('./log/train', sess.graph)
    test_writer = tf.summary.FileWriter('./log/test')


    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.graph.finalize()  # graph is read-only after this statement
    
    # Training loop
    loss_vec = []
    train_acc = []
    test_acc = []
    
    data_train = flatImage(Xtrain, 1, 192171)
    data_test = flatImage(Xval, 1, 6309)
    
    for i in range(3000):
        sess.run(train_step, feed_dict={x: data_train, y: ytrain})
        summary = sess.run(merged, feed_dict={x: data_train, y: ytrain})
        train_writer.add_summary(summary, i)
        summary = sess.run(merged, feed_dict={x: data_test, y: yval})
        test_writer.add_summary(summary, i)



InvalidArgumentError: Incompatible shapes: [192170,1] vs. [192171,1]
	 [[Node: gradients/loss/logistic_loss/mul_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/loss/logistic_loss/mul_grad/Shape, gradients/loss/logistic_loss/mul_grad/Shape_1)]]

Caused by op 'gradients/loss/logistic_loss/mul_grad/BroadcastGradientArgs', defined at:
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/steven/.local/share/virtualenvs/B4-5100-4F18-LSDA-iftAmOns/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/steven/.local/share/virtualenvs/B4-5100-4F18-LSDA-iftAmOns/lib/python3.6/si